<a href="https://colab.research.google.com/github/medinadiegoeverardo/DS-Unit-2-Regression-Classification/blob/master/module2/medinadiego_2_assignment_regression_classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [497]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module2')

Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
From https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification
 * branch            master     -> FETCH_HEAD
Already up to date.


In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv('../data/apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [500]:
df.dtypes

bathrooms               float64
bedrooms                  int64
created                  object
description              object
display_address          object
latitude                float64
longitude               float64
price                     int64
street_address           object
interest_level           object
elevator                  int64
cats_allowed              int64
hardwood_floors           int64
dogs_allowed              int64
doorman                   int64
dishwasher                int64
no_fee                    int64
laundry_in_building       int64
fitness_center            int64
pre-war                   int64
laundry_in_unit           int64
roof_deck                 int64
outdoor_space             int64
dining_room               int64
high_speed_internet       int64
balcony                   int64
swimming_pool             int64
new_construction          int64
terrace                   int64
exclusive                 int64
loft                      int64
garden_p

In [501]:
df.head(5)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [502]:
# exploring data: (to give an idea of which variable may affect y the most? (skewing data))
get_describes = [df[x].describe() for x in df.columns]
print(get_describes)

[count    48817.000000
mean         1.201794
std          0.470711
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max         10.000000
Name: bathrooms, dtype: float64, count    48817.000000
mean         1.537149
std          1.106087
min          0.000000
25%          1.000000
50%          1.000000
75%          2.000000
max          8.000000
Name: bedrooms, dtype: float64, count                   48817
unique                  48148
top       2016-06-11 01:20:36
freq                        3
Name: created, dtype: object, count        47392
unique       37853
top               
freq          1627
Name: description, dtype: object, count        48684
unique        8674
top       Broadway
freq           435
Name: display_address, dtype: object, count    48817.000000
mean        40.750760
std          0.038954
min         40.575700
25%         40.728300
50%         40.751700
75%         40.774000
max         40.989400
Name: latitude, dtype: float64, 

## Splitting

In [0]:
# split data into training and test
am_train = df[(df['created'] > '2016-04-01') & (df['created'] <= '2016-05-30')]
j_test = df[(df['created'] > '2016-06-01') & (df['created'] < '2016-06-30')]

## Mean Baseline

In [504]:
# baseline for y in original df
# initial_baseline = df['price'].mean()
# print('Predicted price ${0:.2f}.'.format(df['price'].mean()))

# baseline for y in april_may_train df
initial_baseline_train = am_train['price'].mean()
print('Predicted price ${0:.2f}.'.format(am_train['price'].mean()))

Predicted price $3575.58.


In [505]:
# train baseline for x effect on y
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# mean absolute error for Train Error
y_train_test = am_train['price'] # split after I come up with feature engineering
y_train_prediction = [initial_baseline_train] * len(y_train_test)
mae = mean_absolute_error(y_train_test, y_train_prediction)
print(f"Our training model's MA error is: ${mae:,.0f}")

Our training model's MA error is: $1,203


In [506]:
# mean absolute error for Test Error
y_test_test = j_test['price'] # split after I come up with feature engineering
y_test_prediction = [initial_baseline_train] * len(y_test_test)
mae_1 = mean_absolute_error(y_test_test, y_test_prediction)
print(f"Our testing model's MA error is: ${mae_1:,.0f}")

Our testing model's MA error is: $1,198


In [507]:
# mean squared error for both Training and Test
squared_train_error = np.sqrt(mean_squared_error(y_train_test, y_train_prediction))
squared_test_error = np.sqrt(mean_squared_error(y_test_test, y_test_prediction))
print(f"Our Training model's squared error is: ${squared_train_error:,.0f}")
print(f"Our Testing model's squared error is: ${squared_test_error:,.0f}")

Our Training model's squared error is: $1,764
Our Testing model's squared error is: $1,763


In [508]:
# r^2 score for both Training and Test
r_training_score = r2_score(y_train_test, y_train_prediction)
r_testing_score = r2_score(y_test_test, y_test_prediction)
print('R-score Training error is: {0:.4f}.'.format(r_training_score))
print('R-score Testing error is: {0:.4f}.'.format(r_testing_score))

R-score Training error is: 0.0000.
R-score Testing error is: -0.0000.


## 1 feature (baseline)

In [509]:
from sklearn.linear_model import LinearRegression
model_1 = LinearRegression()

# features (matrix)
x_train_feature = am_train[['bedrooms']] # why does this need double brackets again?

# target (vector)
y_train_target = am_train['price']

model_1.fit(x_train_feature, y_train_target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [510]:
# coef and intercept
# What do these mean: Coefficient is the slope ('rate per unit'). Intercept is location on y axis
print('Coefficient is {0} \n Intercept is {1}'.format(model_1.coef_, model_1.intercept_))

Coefficient is [856.37384844] 
 Intercept is 2267.2211595160225


In [0]:
#model_1.predict()

def gives_predictions(model, value):
  prediction_values = [[value] for value in len(value)]
  model_predictions = model.predict(prediction_values)
  return model_predictions

  # gives_predictions(model_1, [1, 2, 3])
  # (not working) will come back to this

In [512]:
# just for clarity / visualization
model_predictions = model_1.predict([[1], [2], [3], [4]])
print('1 bedroom prediction {},\n 2 bedroom prediction {},\n 3 bedroom prediction {}'.format(model_predictions[0], model_predictions[1], model_predictions[2]))

1 bedroom prediction 3123.5950079606246,
 2 bedroom prediction 3979.9688564052267,
 3 bedroom prediction 4836.342704849829


In [513]:
# Train Mean absolute error
y_training_pred = model_1.predict(x_train_feature)
mae = mean_absolute_error(y_train_test, y_training_pred) # y_train_test is df['price'] remember
print('On average, we are off by ${0:.2f}'.format(mae))

On average, we are off by $970.19


In [514]:
len(y_training_pred) # just making sure everything looks good. (prediction len has to be the same as y column (am_train['price']))

31626

In [515]:
len(am_train['price'])

31626

## Feature engineering

In [0]:
# I don't use date in the end, ignore

# to date to datetime
# condition = pd.to_datetime(am_train['created'])
# am_train['created'] = condition

In [517]:
# encoding interest level
interest_level_map = {'low': 1, 'medium': 2, 'high':3}
am_train['interest_level'] = am_train['interest_level'].replace(interest_level_map) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [518]:
am_train.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,3,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2.0,4,2016-04-19 04:24:47,,West 18th Street,40.7429,-74.0028,7995,350 West 18th Street,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1.0,2,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,West 107th Street,40.8012,-73.9660,3600,210 West 107th Street,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
am_train_bed_bath = am_train[['bedrooms', 'bathrooms']]
am_train_price = am_train[['price']]

In [0]:
am_train_encoded = pd.get_dummies(am_train_bed_bath, columns=['bedrooms', 'bathrooms'])

In [521]:
am_train_encoded.head()

,bedrooms_0,bedrooms_1,bedrooms_2,bedrooms_3,bedrooms_4,bedrooms_5,bedrooms_6,bedrooms_7,bathrooms_0.0,bathrooms_1.0,bathrooms_1.5,bathrooms_2.0,bathrooms_2.5,bathrooms_3.0,bathrooms_3.5,bathrooms_4.0,bathrooms_4.5,bathrooms_5.0,bathrooms_10.0
2,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
5,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
6,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [522]:
am_train.shape

(31626, 34)

In [523]:
am_train_encoded.shape

(31626, 19)

In [0]:
# # encoding bedrooms
# from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer, OneHotEncoder

# one_hot = LabelBinarizer()
# # to_encode = ['bathrooms', 'bedrooms']
# encoded_bed = one_hot.fit_transform(df[['bedrooms']])
# encoded_bedrooms = pd.DataFrame(encoded_bed)
# #pd.concat([df, encoded_bedrooms], axis=1)

In [0]:
# list comprehension to rename encoded columns

In [526]:
new_df_train = pd.concat([am_train, am_train_encoded], axis=1).drop(columns=['bathrooms', 'bedrooms', 'created', 'description', 'display_address', 'street_address', 'price'])
new_df_train.shape

(31626, 46)

In [527]:
#new_df.drop(columns=['bathrooms', 'bedrooms', 'created', 'description', 'display_address', 'street_address'])
new_df_train.head()

,latitude,longitude,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,bedrooms_0,bedrooms_1,bedrooms_2,bedrooms_3,bedrooms_4,bedrooms_5,bedrooms_6,bedrooms_7,bathrooms_0.0,bathrooms_1.0,bathrooms_1.5,bathrooms_2.0,bathrooms_2.5,bathrooms_3.0,bathrooms_3.5,bathrooms_4.0,bathrooms_4.5,bathrooms_5.0,bathrooms_10.0
2,40.7388,-74.0018,3,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,40.7539,-73.9677,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,40.8241,-73.9493,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
5,40.7429,-74.0028,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
6,40.8012,-73.9660,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [528]:
new_df_train.tail(5)

,latitude,longitude,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,bedrooms_0,bedrooms_1,bedrooms_2,bedrooms_3,bedrooms_4,bedrooms_5,bedrooms_6,bedrooms_7,bathrooms_0.0,bathrooms_1.0,bathrooms_1.5,bathrooms_2.0,bathrooms_2.5,bathrooms_3.0,bathrooms_3.5,bathrooms_4.0,bathrooms_4.5,bathrooms_5.0,bathrooms_10.0
49346,40.7296,-73.9869,2,1,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
49348,40.7102,-74.0163,1,1,1,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
49349,40.7601,-73.9900,1,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
49350,40.7066,-74.0101,1,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
49351,40.8699,-73.9172,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [0]:
# new_df_train.isnull().sum()

#### Note: Notice I have split the data already above using the dates. From here I could've used new_df_train (my training set, it was called am_train BEFORE engineering) for modeling and my j_test (testing set) for predicting and evaluating. Brandon showed me another way to split called 80/20 (manual). This is one way out of others, like the Kfold or using the sklearn train/test func. Since we splitted the data again, I ended up with 8 sets of data. new_df_train, j_test, test_set_from_training, training_set_from_training, AND the last four splits: x_train and y_train from training -- x_test and y_test from training FOR testing/evaluating. If I had done it "correctly" I would've had just 6.

## 2+ features (doing it manually)

### 80/20 split

In [0]:
new_df_train['price'] = am_train[['price']]

In [548]:
new_df_train.shape

(31626, 47)

In [544]:
# training feature split

training_feature_eighty = round(new_df_train[features].shape[0]*.8) 
training_feature_eighty # 80 percent of training data

25301

In [0]:
# will use this to get inverse of 80% training data
ignore_two = list(range(training_feature_eighty))

In [555]:
# 80% training data
print(len(training_set_from_training))
training_set_from_training = new_df_train.loc[new_df_train.index[ignore_two], new_df_train.columns]
training_set_from_training.head()

25301


,latitude,longitude,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,bedrooms_0,bedrooms_1,bedrooms_2,bedrooms_3,bedrooms_4,bedrooms_5,bedrooms_6,bedrooms_7,bathrooms_0.0,bathrooms_1.0,bathrooms_1.5,bathrooms_2.0,bathrooms_2.5,bathrooms_3.0,bathrooms_3.5,bathrooms_4.0,bathrooms_4.5,bathrooms_5.0,bathrooms_10.0,price
2,40.7388,-74.0018,3,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2850
3,40.7539,-73.9677,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3275
4,40.8241,-73.9493,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3350
5,40.7429,-74.0028,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,7995
6,40.8012,-73.9660,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3600


In [554]:
# 20% test data - used 25,301 from previous cell to get the 20% for testing
print(len(test_set_from_training))
test_set_from_training = new_df_train.loc[new_df_train.index[range(25301, new_df_train.shape[0])], new_df_train.columns] # 20 percent of training data
test_set_from_training.head()

6325


,latitude,longitude,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,bedrooms_0,bedrooms_1,bedrooms_2,bedrooms_3,bedrooms_4,bedrooms_5,bedrooms_6,bedrooms_7,bathrooms_0.0,bathrooms_1.0,bathrooms_1.5,bathrooms_2.0,bathrooms_2.5,bathrooms_3.0,bathrooms_3.5,bathrooms_4.0,bathrooms_4.5,bathrooms_5.0,bathrooms_10.0,price
42234,40.7431,-73.9792,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3095
42235,40.8198,-73.9578,2,1,1,1,1,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1950
42236,40.7530,-73.9695,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,4395
42238,40.7436,-73.9725,1,1,1,1,1,1,1,0,0,1,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,4700
42239,40.7323,-73.9531,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1799


In [556]:
len(test_set_from_training + training_set_from_training)

31626

### Modeling

In [557]:
from sklearn.linear_model import LinearRegression
model_2 = LinearRegression()

ignore = 'price'
features = [x for x in new_df_train.columns if x not in ignore]
target = 'price'

# after split and feature engineering

# 80% training portion
x_train = training_set_from_training[features] # x variables (matrix - the model is studying for a test)
y_train = training_set_from_training[target] # what we want to predict (vector)

model_2.fit(x_train, y_train) # studying for test - identifying trends

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

### Predicting

In [0]:
# 20% test portion
x_test = test_set_from_training[features] # NEW questions
y_test = test_set_from_training[target] # Correct answers to test (to compare with model's predictions)

y_pred = model_2.predict(x_test) # predicts on testing data (answers the questions)

In [567]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

r2_s = r2_score(y_test, y_pred) # comparing and evaluating the model's answers test to the correct answer (real data - target)
print('R score is: {}'.format(r2_s))

mse_t = mean_squared_error(y_test, y_pred) # comparing model answers to answers (features)
print('Mean squared error is: ${}'.format(mse_t))

mae_t = mean_absolute_error(y_test, y_pred)
print('Mean absolute error is: ${}'.format(mae_t))

R score is: 0.6567120430321864
Mean squared error is: $1005325.2013993799
Mean absolute error is: $646.1311785929354


### Compared to initial baseline

In [566]:
r_testing_score = r2_score(y_test_test, y_test_prediction)
print('R score is: {0:.4f}.'.format(r_testing_score))

squared_test_error = np.sqrt(mean_squared_error(y_test_test, y_test_prediction))
print(f"Mean squared error is: ${squared_test_error:,.0f}")

mae_1 = mean_absolute_error(y_test_test, y_test_prediction)
print(f"Mean absolute error is: ${mae_1:,.0f}")

R score is: -0.0000.
Mean squared error is: $1,763
Mean absolute error is: $1,198


In [0]:
# not sure what I did wrong with the mean squared error

In [0]:
# ignore

# new_df_train.index (row labels of dataframe 1-n)
# for values in new_df:
#     print(new_df[values].isnull().sum())

- It seems like we need to do feature engineer first, then split. At least when we split manually. Will that change if we use a function like KFold?